# Load functions and packages

In [ ]:
pip install xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 7.8 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import pandas as pd
import xlrd #读取excel数据
import re
import nltk
from nltk.tokenize import MWETokenizer
import re
import sys, time
from matplotlib import pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from scipy.linalg import orthogonal_procrustes
from gensim.models import word2vec
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
def stopwordslist(filepath):
    """
        从文件导入停用词表
        :param filepath: 停用词文件的地址
        :return: 停用词列表
    """
    stopwords=[line.strip() for line in open(filepath,'r',encoding='utf-8').readlines()]
    return stopwords

In [ ]:
# 获取单词的词性
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
def preprocess(doc_set,stopwords):
    """
        处理语料（数据清洗+分词+去停用词）
        :param doc_set: 语料库的数据，类型为list
        :param stopwords: 停用词列表
        :return: 处理好的文章关键词
    """
    texts = []#每篇文章关键词
    for doc in doc_set:
        doc=doc.lower()
        # 数据清洗，删除数字以及特殊符号
        cop=re.compile('\d')
        cleaned_doc=cop.sub('',doc)
        cop=re.compile('[.:/()\?!\(\)\[\]\{\}\*\+\\\|\"\<\>,~@#$%^&\-+=;\'\`–“”—_·]')
        cleaned_doc=cop.sub('',cleaned_doc)
        # 小写化
        cleaned_doc=cleaned_doc.lower()
        # 分词
        doc_cut=nltk.word_tokenize(cleaned_doc)
        # print(doc_cut)
        # 获取词的词性后进行词形还原
        tagged_sent = pos_tag(doc_cut)
        wnl = WordNetLemmatizer()
        lemmas_sent = [] # 词形还原后的结果
        for tag in tagged_sent:
          wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
          lemmas_sent.append(wnl.lemmatize(tag[0], pos=wordnet_pos)) # 词形还原
        # 词干提取
        # ps = PorterStemmer()
        # porter = []
        # for w in lemmas_sent:
        #   porter.append(ps.stem(w))
        # cleaned_doc=porter_stemmer.stem(cleaned_doc)
        # 去停用词
        text_list = [word for word in lemmas_sent if word not in stopwords and len(word)>2 and len(word)<25]
        # print(text_list)
        texts.append(text_list)
    return texts

In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
stopwords = stopwordslist(r"stopwords_English+Latin.txt")

# Train Word2Vec

## 1570-1620

In [ ]:
year = "1570-1620"
doc_set = []
workbook = xlrd.open_workbook("lineage, variant, clade, strain_allData.xlsx")
sheet = workbook.sheet_by_name(year)
for index in range(sheet.nrows):
  doc_set.append(sheet.row_values(index)[0])
texts = preprocess(doc_set, stopwords)
# print(texts)
t=''
j=0
with open (year+'.txt','w') as q:
    for i in texts:
      for e in range(len(texts[j])):
        t=t+str(i[e])+' '
      j+=1
      q.write(t.strip(' '))
      q.write('\n')
      t=''
sentences = word2vec.Text8Corpus(year+'.txt')
model_1500 = word2vec.Word2Vec(sentences=sentences, size=100, window = 8, iter = 10) # CBOW算法
# window = 8：控制窗口，如果设得较小，那么模型学习到的是词汇间的组合性关系（词性相异）；如果设置得较大，会学习到词汇之间的聚合性关系（词性相同）。模型默认的window数值为5
# size = 100：size是输出词向量的维数，即神经网络的隐藏层的单元数
# iter = 10：假如参与训练的文本量较少，就需要把这个参数调大一些
model_1500.save('word2vec_1570-1620.bin')

## 1670-1720

In [ ]:
year = "1670-1720"
doc_set = []
workbook = xlrd.open_workbook("lineage, variant, clade, strain_allData.xlsx")
sheet = workbook.sheet_by_name(year)
for index in range(sheet.nrows):
  doc_set.append(sheet.row_values(index)[0])
texts = preprocess(doc_set, stopwords)
# print(texts)
t=''
j=0
with open (year+'.txt','w') as q:
    for i in texts:
      for e in range(len(texts[j])):
        t=t+str(i[e])+' '
      j+=1
      q.write(t.strip(' '))
      q.write('\n')
      t=''
from gensim.models import word2vec
sentences = word2vec.Text8Corpus(year+'.txt')
model_1540 = word2vec.Word2Vec(sentences=sentences, size=100, window = 8, iter = 10)
model_1540.save('word2vec_1670-1720.bin')

## 1770-1820

In [ ]:
year = "1770-1820"
doc_set = []
workbook = xlrd.open_workbook("lineage, variant, clade, strain_allData.xlsx")
sheet = workbook.sheet_by_name(year)
for index in range(sheet.nrows):
  doc_set.append(sheet.row_values(index)[0])
texts = preprocess(doc_set, stopwords)
# print(texts)
t=''
j=0
with open (year+'.txt','w') as q:
    for i in texts:
      for e in range(len(texts[j])):
        t=t+str(i[e])+' '
      j+=1
      q.write(t.strip(' '))
      q.write('\n')
      t=''
from gensim.models import word2vec
sentences = word2vec.Text8Corpus(year+'.txt')
model_1610 = word2vec.Word2Vec(sentences=sentences, size=100, window = 8, iter = 10)
model_1610.save('word2vec_1770-1820.bin')

## 1870-1920

In [ ]:
year = "1870-1920"
doc_set = []
workbook = xlrd.open_workbook("lineage, variant, clade, strain_allData.xlsx")
sheet = workbook.sheet_by_name(year)
for index in range(sheet.nrows):
  doc_set.append(sheet.row_values(index)[0])
texts = preprocess(doc_set, stopwords)
# print(texts)
t=''
j=0
with open (year+'.txt','w') as q:
    for i in texts:
      for e in range(len(texts[j])):
        t=t+str(i[e])+' '
      j+=1
      q.write(t.strip(' '))
      q.write('\n')
      t=''
from gensim.models import word2vec
sentences = word2vec.Text8Corpus(year+'.txt')
model_1680 = word2vec.Word2Vec(sentences=sentences, size=100, window = 8, iter = 10)
model_1680.save('word2vec_1870-1920.bin')

## 1970-2020

In [ ]:
year = "1970-2020"
doc_set = []
workbook = xlrd.open_workbook("lineage, variant, clade, strain_allData.xlsx")
sheet = workbook.sheet_by_name(year)
for index in range(sheet.nrows):
  doc_set.append(sheet.row_values(index)[0])
texts = preprocess(doc_set, stopwords)
# print(texts)
t=''
j=0
with open (year+'.txt','w') as q:
    for i in texts:
      for e in range(len(texts[j])):
        t=t+str(i[e])+' '
      j+=1
      q.write(t.strip(' '))
      q.write('\n')
      t=''
from gensim.models import word2vec
sentences = word2vec.Text8Corpus(year+'.txt')
model_1750 = word2vec.Word2Vec(sentences=sentences, size=100, window = 8, iter = 10)
model_1750.save('word2vec_1970-2020.bin')

# Save model

In [ ]:
import xlwt
def save_file(year,words):
  workbook = xlwt.Workbook(encoding = 'ascii')  
  model = word2vec.Word2Vec.load('word2vec_'+year+'.bin')
  for word in words:
    row = 0
    worksheet = workbook.add_sheet(word)
    worksheet.write(row, 0, "word")
    worksheet.write(row, 1, "similarity")
    row+=1
    for i in model.most_similar(word,topn=500):      
      # 保存数据前相似的
      worksheet.write(row, 0, i[0]) # 相似词
      worksheet.write(row, 1, i[1]) # 相似概率
      row+=1
    workbook.save('similarity_'+year+'.xls')  # 保存文件

In [ ]:
# 归一化后再存入
import xlwt
def MaxMinNormalization(x,Max,Min):
    x = 100*(x - Min) / (Max - Min); #归一化到0-100
    return x;
def save_file(year,words):
  workbook = xlwt.Workbook(encoding = 'ascii')  
  model = word2vec.Word2Vec.load('word2vec_'+year+'.bin')
  for word in words:
    row = 0
    worksheet = workbook.add_sheet(word)
    worksheet.write(row, 0, "word")
    worksheet.write(row, 1, "similarity")
    worksheet.write(row, 2, "normalized")
    row+=1
    first = 0
    max = 0
    min = 0
    for i in model.most_similar(word,topn=200):
      # 保存数据前相似的
      worksheet.write(row, 0, i[0]) # 相似词
      worksheet.write(row, 1, i[1]) # 相似概率
      row+=1
      if(first==0):
        max=i[1]
        min=i[1]
        first=1
      else:
        if(i[1]>max):
          max=i[1]
        if(i[1]<min):
          min=i[1]
    normal_row = 1
    for i in model.most_similar(word,topn=50):
      worksheet.write(normal_row, 2
                , round(MaxMinNormalization(i[1],max,min),2)) # 归一化
      normal_row+=1
    workbook.save('similarity_'+year+'.xls')  # 保存文件

In [ ]:
save_file("1670-1720",["variant","clade","strain"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
save_file("1770-1820",["lineage","variant","clade","strain"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
save_file("1870-1920",["lineage","variant","clade","strain"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
save_file("1970-2020",["lineage","variant","clade","strain"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


# Load model

In [ ]:
model_1 = word2vec.Word2Vec.load('word2vec_1970-2020.bin')
for i in model_1.most_similar("lineage",topn=40):
  print(i[0],i[1])

In [ ]:
model_2 = word2vec.Word2Vec.load('word2vec_1870-1920.bin')
for i in model_2.most_similar("variation",topn=40):
  print(i[0],i[1])

In [ ]:
model_3 = word2vec.Word2Vec.load('word2vec_1770-1820.bin')
for i in model_3.most_similar("agros",topn=100):
  print(i[0],i[1])

In [ ]:
model_4 = word2vec.Word2Vec.load('word2vec_1670-1720.bin')
for i in model_4.most_similar("venenum",topn=40):
  print(i[0],i[1])

In [ ]:
model_5 = word2vec.Word2Vec.load('word2vec_1570-1620.bin')
for i in model_5.most_similar("religionis",topn=40):
  print(i[0],i[1])